In [ ]:
# HIDDEN
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np

## Lecture 23 ##

In [ ]:
x = make_array(43, 20, 51, 7, 28, 34)
y = np.sort(x)
y

In [ ]:
.55*6

In [ ]:
percentile(55, x)

In [ ]:
percentile(50, x)

In [ ]:
0.95 * 6

In [ ]:
percentile(95, x)

In [ ]:
sf = Table.read_table('san_francisco_2015.csv')

In [ ]:
sf

In [ ]:
sf.where('Job', are.equal_to('Mayor'))

In [ ]:
sf.sort('Total Compensation')

In [ ]:
10*20*52

In [ ]:
sf = sf.where('Salaries', are.above(10000))

In [ ]:
sf.num_rows

In [ ]:
sf_bins = np.arange(0, 700000, 25000)
sf.hist('Total Compensation', bins = sf_bins)

In [ ]:
sf.sort('Total Compensation', descending=True).show(5)

In [ ]:
pop_median = percentile(50, sf.column('Total Compensation'))
pop_median

In [ ]:
our_sample = sf.sample(300, with_replacement=False)

In [ ]:
our_sample.hist('Total Compensation', bins = sf_bins)

In [ ]:
est_median = percentile(50, our_sample.column('Total Compensation'))
est_median

In [ ]:
resample_1 = our_sample.sample()

In [ ]:
resample_1.hist('Total Compensation', bins = sf_bins)

In [ ]:
resampled_median_1 = percentile(50, resample_1.column('Total Compensation'))
resampled_median_1

In [ ]:
resample_2 = our_sample.sample()
resampled_median_2 = percentile(50, resample_2.column('Total Compensation'))
resampled_median_2

In [ ]:
def bootstrap_median(original_sample, label, replications):
    """Simulate sample median:
    original_sample: table containing the original sample
    label: label of column containing the variable
    replications: number of bootstrap samples
    Returns array of bootstrap sample medians
    """
    medians = make_array()
    for i in np.arange(replications):
        bootstrap_sample = original_sample.sample()
        resampled_median = percentile(50, bootstrap_sample.column(label))
        medians = np.append(medians, resampled_median)
        
    return medians

In [ ]:
bstrap_medians = bootstrap_median(our_sample, 'Total Compensation', 2000)

In [ ]:
resampled_medians = Table().with_column(
    'Bootstrap Sample Median', bstrap_medians)

resampled_medians.hist()

plots.scatter(pop_median, 0, color='red', s=40);

In [ ]:
left = percentile(2.5, bstrap_medians)
left

In [ ]:
right = percentile(97.5, bstrap_medians)
right

In [ ]:
resampled_medians.hist()

plots.plot([left, right], [0, 0], color='yellow', lw=3, zorder=1)
plots.scatter(pop_median, 0, color='red', s=30, zorder=2);

In [ ]:
# THE BIG SIMULATION: This one takes a long time.

# Generate the endpoints of 50 intervals

left_ends = make_array()
right_ends = make_array()

for i in np.arange(50):
    first_sample = sf.sample(300, with_replacement=False)
    medians = bootstrap_median(first_sample, 'Total Compensation', 2000)
    left_ends = np.append(left_ends, percentile(2.5, medians))
    right_ends = np.append(right_ends, percentile(97.5, medians))

In [ ]:
intervals = Table().with_column(
    'Left', left_ends,
    'Right', right_ends
)    

In [ ]:
intervals

In [ ]:
good = intervals.where('Left', are.below(pop_median)).where('Right', are.above(pop_median)).num_rows

In [ ]:
good / 50